In [1]:
import pandas as pd
import pyspark.sql
from pyspark.sql import SparkSession
import random
import time
import findspark
import pyspark
import pandas as pd
import pyspark.sql
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
%matplotlib inline
from pyspark.ml.feature import CountVectorizer

In [2]:
!pip install pandas

In [3]:
findspark.init(r"C:\Users\rejee\spark-3.3.1-bin-hadoop3")

In [4]:
spark = SparkSession.builder.appName("BDA").getOrCreate()

In [5]:
spark

In [6]:
df = spark.read.load(".\kindle_reviews.csv", format="csv", sep=",", header="true", inferSchema=True)

In [7]:
df.show()

+---+----------+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|_c0|      asin|helpful|overall|          reviewText|          reviewTime|          reviewerID|        reviewerName|             summary|      unixReviewTime|
+---+----------+-------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|  0|B000F83SZQ| [0, 0]|      5|I enjoy vintage b...|          05 5, 2014|      A1F6404F1VG29J|          Avidreader|  Nice vintage story|          1399248000|
|  1|B000F83SZQ| [2, 2]|      4|This book is a re...|          01 6, 2014|       AN0N05A9LIJEQ|            critters|        Different...|          1388966400|
|  2|B000F83SZQ| [2, 2]|      4|This was a fairly...|          04 4, 2014|       A795DMNCJILA6|                 dot|               Oldie|          1396569600|
|  3|B000F83SZQ| [1, 1]|      5|I'd never read

In [8]:
df=df.drop('reviewTime','reviewerName','reviewerID','helpful','unixReviewTime')

In [9]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- asin: string (nullable = true)
 |-- overall: integer (nullable = true)
 |-- reviewText: string (nullable = true)
 |-- summary: string (nullable = true)



In [10]:
df.head(3)

[Row(_c0='0', asin='B000F83SZQ', overall=5, reviewText="I enjoy vintage books and movies so I enjoyed reading this book.  The plot was unusual.  Don't think killing someone in self-defense but leaving the scene and the body without notifying the police or hitting someone in the jaw to knock them out would wash today.Still it was a good read for me.", summary='Nice vintage story'),
 Row(_c0='1', asin='B000F83SZQ', overall=4, reviewText="This book is a reissue of an old one; the author was born in 1910. It's of the era of, say, Nero Wolfe. The introduction was quite interesting, explaining who the author was and why he's been forgotten; I'd never heard of him.The language is a little dated at times, like calling a gun a &#34;heater.&#34;  I also made good use of my Fire's dictionary to look up words like &#34;deshabille&#34; and &#34;Canarsie.&#34; Still, it was well worth a look-see.", summary='Different...'),
 Row(_c0='2', asin='B000F83SZQ', overall=4, reviewText="This was a fairly int

In [11]:
df.show()

+---+----------+-------+--------------------+--------------------+
|_c0|      asin|overall|          reviewText|             summary|
+---+----------+-------+--------------------+--------------------+
|  0|B000F83SZQ|      5|I enjoy vintage b...|  Nice vintage story|
|  1|B000F83SZQ|      4|This book is a re...|        Different...|
|  2|B000F83SZQ|      4|This was a fairly...|               Oldie|
|  3|B000F83SZQ|      5|I'd never read an...|  I really liked it.|
|  4|B000F83SZQ|      4|If you like perio...|      Period Mystery|
|  5|B000F83SZQ|      4|A beautiful in-de...|              Review|
|  6|B000F83SZQ|      4|I enjoyed this on...|Nice old fashione...|
|  7|B000F83SZQ|      4|Never heard of Am...|Enjoyable reading...|
|  8|B000FA64PA|      5|Darth Maul workin...|          Darth Maul|
|  9|B000FA64PA|      4|This is a short s...|Not bad, not exce...|
| 10|B000FA64PA|      5|I think I have th...|      Audio and book|
| 11|B000FA64PA|      4|Title has nothing...|Darth Maul...the 

In [12]:
df.select('summary').show()

+--------------------+
|             summary|
+--------------------+
|  Nice vintage story|
|        Different...|
|               Oldie|
|  I really liked it.|
|      Period Mystery|
|              Review|
|Nice old fashione...|
|Enjoyable reading...|
|          Darth Maul|
|Not bad, not exce...|
|      Audio and book|
|Darth Maul...the ...|
|Not bad; it is we...|
|Han and Leia reun...|
|  Possibly Important|
|        Another read|
|            Recovery|
|Star Wars: The Ne...|
|       my collection|
| where it's bille...|
+--------------------+
only showing top 20 rows



In [13]:
df.select(['summary','overall']).show()

+--------------------+-------+
|             summary|overall|
+--------------------+-------+
|  Nice vintage story|      5|
|        Different...|      4|
|               Oldie|      4|
|  I really liked it.|      5|
|      Period Mystery|      4|
|              Review|      4|
|Nice old fashione...|      4|
|Enjoyable reading...|      4|
|          Darth Maul|      5|
|Not bad, not exce...|      4|
|      Audio and book|      5|
|Darth Maul...the ...|      4|
|Not bad; it is we...|      3|
|Han and Leia reun...|      3|
|  Possibly Important|      5|
|        Another read|      5|
|            Recovery|      5|
|Star Wars: The Ne...|      3|
|       my collection|      5|
| where it's bille...|      4|
+--------------------+-------+
only showing top 20 rows



In [14]:
df.describe().show()

+-------+----------------+--------------------+------------------+--------------------+--------------------+
|summary|             _c0|                asin|           overall|          reviewText|             summary|
+-------+----------------+--------------------+------------------+--------------------+--------------------+
|  count|          982820|              982815|            982619|              982597|              982385|
|   mean|        491309.0|                null| 4.347801131466011|                null|            Infinity|
| stddev|283657.816417951|                null|0.9550557821749521|                null|                 NaN|
|    min|               0| 30 Prayers Of Peace|                 1| &#10032; Bianca ...|                    |
|    max|             wn"|looking forward t...|                 5|~~~~~~~~~~~~~Doub...|~~~~~~~His forbid...|
+-------+----------------+--------------------+------------------+--------------------+--------------------+



In [15]:
#df=df.drop('reviewTime','reviewerName','reviewerID','helpful','unixReviewTime')

In [16]:
df.show()

+---+----------+-------+--------------------+--------------------+
|_c0|      asin|overall|          reviewText|             summary|
+---+----------+-------+--------------------+--------------------+
|  0|B000F83SZQ|      5|I enjoy vintage b...|  Nice vintage story|
|  1|B000F83SZQ|      4|This book is a re...|        Different...|
|  2|B000F83SZQ|      4|This was a fairly...|               Oldie|
|  3|B000F83SZQ|      5|I'd never read an...|  I really liked it.|
|  4|B000F83SZQ|      4|If you like perio...|      Period Mystery|
|  5|B000F83SZQ|      4|A beautiful in-de...|              Review|
|  6|B000F83SZQ|      4|I enjoyed this on...|Nice old fashione...|
|  7|B000F83SZQ|      4|Never heard of Am...|Enjoyable reading...|
|  8|B000FA64PA|      5|Darth Maul workin...|          Darth Maul|
|  9|B000FA64PA|      4|This is a short s...|Not bad, not exce...|
| 10|B000FA64PA|      5|I think I have th...|      Audio and book|
| 11|B000FA64PA|      4|Title has nothing...|Darth Maul...the 

In [17]:
df=df.withColumnRenamed('overall','Rating')

In [18]:
df.show()

+---+----------+------+--------------------+--------------------+
|_c0|      asin|Rating|          reviewText|             summary|
+---+----------+------+--------------------+--------------------+
|  0|B000F83SZQ|     5|I enjoy vintage b...|  Nice vintage story|
|  1|B000F83SZQ|     4|This book is a re...|        Different...|
|  2|B000F83SZQ|     4|This was a fairly...|               Oldie|
|  3|B000F83SZQ|     5|I'd never read an...|  I really liked it.|
|  4|B000F83SZQ|     4|If you like perio...|      Period Mystery|
|  5|B000F83SZQ|     4|A beautiful in-de...|              Review|
|  6|B000F83SZQ|     4|I enjoyed this on...|Nice old fashione...|
|  7|B000F83SZQ|     4|Never heard of Am...|Enjoyable reading...|
|  8|B000FA64PA|     5|Darth Maul workin...|          Darth Maul|
|  9|B000FA64PA|     4|This is a short s...|Not bad, not exce...|
| 10|B000FA64PA|     5|I think I have th...|      Audio and book|
| 11|B000FA64PA|     4|Title has nothing...|Darth Maul...the ...|
| 12|B000F

In [19]:
df.show(5)

+---+----------+------+--------------------+------------------+
|_c0|      asin|Rating|          reviewText|           summary|
+---+----------+------+--------------------+------------------+
|  0|B000F83SZQ|     5|I enjoy vintage b...|Nice vintage story|
|  1|B000F83SZQ|     4|This book is a re...|      Different...|
|  2|B000F83SZQ|     4|This was a fairly...|             Oldie|
|  3|B000F83SZQ|     5|I'd never read an...|I really liked it.|
|  4|B000F83SZQ|     4|If you like perio...|    Period Mystery|
+---+----------+------+--------------------+------------------+
only showing top 5 rows



In [20]:
grouped_df = df.groupBy("Rating")

In [21]:
counts = grouped_df.count()

counts.show()

+------+------+
|Rating| count|
+------+------+
|  null|   201|
|     1| 23018|
|     3| 96194|
|     5|575264|
|     4|254013|
|     2| 34130|
+------+------+



In [22]:
df=df.dropna()

In [23]:
import pyspark.ml.feature

In [24]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import Tokenizer,StopWordsRemover,IDF,CountVectorizer
from pyspark.ml.feature import StringIndexer,OneHotEncoder

In [25]:
tokenizer=Tokenizer(inputCol='summary',outputCol='review_token')
#stop_words=StopWordsRemover(inputCol='review_token',outputCol='review_stop')
countvector=CountVectorizer(inputCol='review_token',outputCol='review_count')

In [26]:
indexer = StringIndexer(inputCol='Rating', outputCol='label')

In [27]:
#indexer.transform(df).show(20)

In [28]:
from pyspark.ml import Pipeline

In [29]:
pipeline = Pipeline(stages=[tokenizer,countvector,indexer ])

In [30]:
Fit = pipeline.fit(df)
dataset = Fit.transform(df)

In [ ]:
############

In [31]:
[trainingData, testData] = df.randomSplit([0.7, 0.3])

In [32]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [33]:
lr = LogisticRegression(maxIter = 50, regParam = 0.01)

In [62]:
pipeline = Pipeline(stages=[tokenizer,countvector, indexer, lr])

In [63]:
# Building MOdel
lr_model = pipeline.fit(trainingData)


IllegalArgumentException: Idf_features does not exist. Available: _c0, asin, Rating, reviewText, summary, review_token, review_count, label

In [35]:
lr_model

PipelineModel_e568cd5dbb1a

In [64]:
predictions = lr_model.transform(testData)

In [65]:
predictions.show()

+------+----------+------+--------------------+--------------------+--------------------+--------------------+-----+
|   _c0|      asin|Rating|          reviewText|             summary|        review_token|        review_count|label|
+------+----------+------+--------------------+--------------------+--------------------+--------------------+-----+
|100003|B005HAWAZG|     5|I love the Myrtle...|    Simply enjoyable| [simply, enjoyable]|(148303,[73,434],...|  0.0|
|100004|B005HAWAZG|     4|Myrtle Clover, a ...|A quirky who-done...|[a, quirky, who-d...|(148303,[0,906],[...|  1.0|
|100009|B005HAWAZG|     2|This book was ver...|     Confusing story|  [confusing, story]|(148303,[14,788],...|  3.0|
|100014|B005HAWAZG|     4|SPOILER ALERT! Th...|Good book but som...|[good, book, but,...|(148303,[5,9,20,9...|  1.0|
|100017|B005HAWAZG|     5|I love these book...|           Excellent|         [excellent]| (148303,[49],[1.0])|  0.0|
|100018|B005HAWAZG|     5|I just finished r...|     A Good Read!

In [66]:
predictions.columns

['_c0',
 'asin',
 'Rating',
 'reviewText',
 'summary',
 'review_token',
 'review_count',
 'label']

In [61]:
predictions.select('summary','Rating','label','prediction').show(10)

AnalysisException: Column 'prediction' does not exist. Did you mean one of the following? [Rating, asin, reviewText, review_count, review_token, _c0, label, summary];
'Project [summary#25, Rating#571, label#917, 'prediction]
+- Project [_c0#17, asin#18, Rating#571, reviewText#21, summary#25, review_token#890, review_count#903, UDF(cast(Rating#571 as string)) AS label#917]
   +- Project [_c0#17, asin#18, Rating#571, reviewText#21, summary#25, review_token#890, UDF(review_token#890) AS review_count#903]
      +- Project [_c0#17, asin#18, Rating#571, reviewText#21, summary#25, UDF(summary#25) AS review_token#890]
         +- Sample 0.7, 1.0, false, 622724739664976348
            +- Sort [_c0#17 ASC NULLS FIRST, asin#18 ASC NULLS FIRST, Rating#571 ASC NULLS FIRST, reviewText#21 ASC NULLS FIRST, summary#25 ASC NULLS FIRST], false
               +- Filter atleastnnonnulls(5, _c0#17, asin#18, Rating#571, reviewText#21, summary#25)
                  +- Project [_c0#17, asin#18, overall#20 AS R

In [51]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [59]:
evaluator = MulticlassClassificationEvaluator(labelCol='label',predictionCol='prediction',metricName='accuracy')

In [60]:

accuracy = evaluator.evaluate(predictions)

IllegalArgumentException: prediction does not exist. Available: _c0, asin, Rating, reviewText, summary, review_token, review_count, label